In [308]:
#import packages
import numpy as np
import pandas as pd
import cv2
import time
import math
cv2.startWindowThread()


0

In [309]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/colorblock_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps
print(fps)
if cap.isOpened() == False:
    print("Cannot open video file.")

119.88011988011988


In [310]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)


In [311]:
#defining size for output videos
output = cv2.VideoWriter('/Users/danielrosenthal/Documents/EGR_314/colorblock_output2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (1920,1080))
#output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

In [312]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [313]:
#creating empty array
# Create an empty DataFrame with columns for x and y velocities
velocities = pd.DataFrame(columns=['x_velocity', 'y_velocity'])
prev_x, prev_y = None, None
prev_time = None


In [314]:
#defining rgb range to detect orange for now
lower_color_bound = np.array([0,100,178])
upper_color_bound = np.array([255,255,255])

blue_lower_bound = np.array([100,80,80])
blue_upper_bound = np.array([170,255,255])

yellow_lower_bound = np.array([30, 100, 100])
yellow_upper_bound = np.array([80, 255, 255])

In [315]:
#conversion factor, likely incorrect bc camera and setup need to be calibrated
disc_length = 8.26 #disc length in inches
distance = 36 #distance of disc from camera, rough guess

#initializing
conversion_rate = 0
rect_coordinates = 0
new_data = None
counter = 0


In [316]:
prev_frame = None
curr_frame = None
threshold = 500
x = 0
y = 0
centers_and_labels = {}
prev_centers_and_labels = {}
distance_threshold = 50

In [317]:
while True:
    # Get current time
    current_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break
    
    height, width, _ = frame.shape
    
    
    #object detection
    mask = object_detector.apply(frame)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    #trying rgb threshold
    mask = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) 
    mask_blue = cv2.inRange(mask,blue_lower_bound,blue_upper_bound)
    mask_yellow = cv2.inRange(mask,yellow_lower_bound, yellow_upper_bound)
    
    yellow_contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    blue_contours, _ = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    large_contours = [cnt for cnt in blue_contours if cv2.contourArea(cnt) > threshold]
    
    mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    contour_labels = []

    longest_width = 0
    for cnt in yellow_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(frame, [cnt], -1, (255,0,255), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            if w > longest_width:
                longest_width = w
            #print(longest_width)
            rect_coordinates = np.append(rect_coordinates,[x,y,w,h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255,0,255),3)
                
    
            
    if longest_width > 0:
        conversion_rate = disc_length / longest_width
    

    # Check if previous position and time are available
    if prev_x is not None and prev_y is not None and prev_time is not None:
        # Calculate the distance traveled in the x and y directions
        dx = x - prev_x
        dy = y - prev_y

        # Calculate the time interval
        dt = current_time - prev_time

        # Calculate the velocity in the x and y directions
        vx = (dx / dt) 
        vy = (dy / dt)
        
        # Add the velocity values to the DataFrame
        velocities = pd.concat([velocities, new_data])

        vx_mph = vx * conversion_rate
        vy_mph = vy * conversion_rate
        velocity = np.linalg.norm([vx_mph, vy_mph])

        # Display the velocity values on the output video
        cv2.putText(frame, 'X Velocity: {:.2f} mph'.format(vx_mph), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'Y Velocity: {:.2f} mph'.format(vy_mph), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'True Velocity: {:.2f} mph'.format(velocity), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # create a new dataframe with the new velocities
        new_data = pd.DataFrame({'x_velocity': [vx_mph], 'y_velocity': [vy_mph]})









# everything below here is trying to calculate spin rate

    # Find the center of each contour
    centers = []
    prev_center = None
    for cnt in large_contours:
        M = cv2.moments(cnt)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centers.append([cX, cY])
        
    # If there are no centers, break out of the loop
    # if not centers:
    #     break

    # Assign labels to the centers in the current frame
    if prev_centers_and_labels == {}:
        # First frame, assign labels to all centers
        for j in range(len(centers)):
            centers_and_labels[j] = [centers[j], j + 1]
    else:
        # Not the first frame, assign labels based on previous centers
        for j in range(len(centers)):
            closest_center = None
            closest_center_distance = float('inf')
            closest_center_label = None
            # Find the closest previous center
            for center_index, previous_center_and_label in prev_centers_and_labels.items():
                previous_center, previous_center_label = previous_center_and_label
                distance = np.sqrt((previous_center[0] - centers[j][0]) ** 2 + (previous_center[1] - centers[j][1]) ** 2)
                if distance < closest_center_distance:
                    closest_center = previous_center
                    closest_center_distance = distance
                    closest_center_label = previous_center_label
            # Assign the same label to the current center if it's close enough
            if closest_center_distance < distance_threshold:
                centers_and_labels[j] = [centers[j], closest_center_label]
            else:
                # If no close enough previous center was found, assign a new label
                centers_and_labels[j] = [centers[j], max(prev_centers_and_labels.values(), key=lambda x: x[1])[1] + 1]

    for i in range(len(large_contours)):
        x, y, w, h = cv2.boundingRect(large_contours[i])
        cv2.putText(frame, str(centers_and_labels[i][1]), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
        # cv2.line(frame, tuple(centers_and_labels[i][1]), tuple(prev_centers_and_labels[i][0]), (255, 0, 0), 2)
        cv2.drawContours(frame, [large_contours[i]], 0, (0, 255, 255), 2)
        
   
    # Update the previous position and time variables
    prev_x, prev_y = x, y
    prev_time = current_time

    cv2.imshow("frame", frame)
    # cv2.imshow("frame keypoints", frame_keypoints)
    #cv2.imshow("roi", roi)
    #cv2.imshow("Mask", mask)
    # cv2.imshow("mask yellow",mask_yellow)
    
    output.write(frame)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break


cap.release()
output.release()
#output2.release()
cv2.destroyAllWindows()

In [318]:
velocities.to_csv("velocities.csv", index=False)
print(velocities)

   x_velocity  y_velocity
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000  444.721670
0    2.897137   -0.724284
0   35.932844  -11.056260
0   30.167933   -8.379981
0   23.127696   -6.802263
0   20.775154   -5.770876
0   22.909330   -4.749495
0   21.314204   -5.074810
0   23.034379   -9.161401
0    3.343535   -7.164717
0    4.651946   -4.651946
0    9.595890   -4.545422
0   15.814965   -4.447959
0   18.745551   -4.271138
0   21.017918   -4.154705
0   19.653739   -6.785219
0   -0.921122   -8.980940
0   -0.836648   -5.019885
0    3.653529   -5.219327
0   28.837551   -7.864787
